In [44]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [45]:
image_file = "data/test_img.png"
image = "data/test_img.png"
img = cv2.imread(image_file)

In [46]:
img.shape

(2339, 1653, 3)

In [47]:
#https://stackoverflow.com/questions/28816046/
#displaying-different-images-with-actual-size-in-matplotlib-subplot
def display(im_path):
    dpi = 80
    im_data = plt.imread(im_path)

    height, width  = im_data.shape[:2]
    
    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, cmap='gray')

    plt.show()

In [48]:
# display(image_file)

In [49]:
# Grayscale conversion
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [50]:
gray_image = grayscale(img)
cv2.imwrite("temp/gray.jpg", gray_image)

True

In [51]:
gray_image.shape

(2339, 1653)

## Binarize

In [52]:
thresh, im_bw = cv2.threshold(gray_image, 200,230, cv2.THRESH_BINARY)
cv2.imwrite("temp/bw_image.jpg",im_bw)

True

In [53]:
print(thresh)

200.0


In [54]:
# display("temp/bw_image.jpg")

## Bounding Boxes

In [55]:
cnts = cv2.findContours(im_bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cents[1]
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])
print(cnts)

[array([[[   0,    0]],

       [[   0, 2338]],

       [[1652, 2338]],

       [[1652,    0]]], dtype=int32)]


In [56]:
 # Find contours
contours, _ = cv2.findContours(im_bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Get bounding boxes around each contour
bounding_boxes = [cv2.boundingRect(contour) for contour in contours]
print(bounding_boxes)
# Draw bounding boxes
for x, y, w, h in bounding_boxes:
    img_bbox = cv2.rectangle(im_bw, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imwrite("temp/bbox.png",img_bbox)

[(0, 0, 1653, 2339)]


True

In [57]:
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    # if h > 200 and w > 20:
    roi = im_bw[y:y+h, x:x+h]
    cv2.imwrite("temp/index_roi.png", roi)
    cv2.rectangle(im_bw, (x, y), (x+w, y+h), (36, 255, 12), 2)
cv2.imwrite("temp/index_bbox_new.png", im_bw)

True

## Italics detection

In [58]:
# https://stackoverflow.com/questions/67577793/detecting-bold-and-italic-text-in-an-image
KERNEL = np.asarray([
    [1, 1, 1, 1],
    [1, 1, 1, 1],
    [1, 1, 1, 1],
], np.uint8)
KERNEL_ITALIC = np.asarray([
    [0, 0, 1, 1],
    [0, 0, 1, 1],
    [0, 0, 1, 1],
    [0, 1, 1, 0],
    [0, 1, 1, 0],
    [0, 1, 1, 0],
    [1, 1, 0, 0],
    [1, 1, 0, 0],
    [1, 1, 0, 0],
], np.uint8)

def pre_process_italic(img):
    img_f = cv2.flip(img, 1)

    img = cv2.erode(img, KERNEL_ITALIC, iterations=1)
    img = cv2.dilate(img, KERNEL, iterations=1)

    img_f = cv2.erode(img_f, KERNEL_ITALIC, iterations=1)
    img_f = cv2.dilate(img_f, KERNEL, iterations=1)
    img_f = cv2.flip(img_f, 1)
    return img, img_f

In [59]:
def apply_func_italic(bbox, original, preprocessed):

    b_1 = bbox[1]
    b_3 = bbox[3]
    b_0 = bbox[0]
    b_2 = bbox[2]

    a, b = np.mean(original[b_1:b_3, b_0:b_2]), np.mean(preprocessed[b_1:b_3, b_0:b_2])

    return get_ratio(a, b)

def get_ratio(a, b):
    return ((a - b) / (a + b + 1e-8)) * 2

In [60]:
img, img_f = pre_process_italic(im_bw)
ratio_1 = apply_func_italic(cnts, im_bw, img)
ratio_2 = apply_func_italic(cnts, im_bw, img_f)